# But : Créer un masque sans végétation avec une image infrarouge Pléiade.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
! pip install rasterio  -q -q -q
! pip install geopandas -q -q -q
! pip install matplotlib -q -q -q

In [3]:
!pip install pyarrow
!pip install opencv-python
!pip install os-sys

In [37]:
import sys
sys.path.append('../src')
from satellite_image import SatelliteImage
from utils import *
from plot_utils import *

Lancer ça dans le terminal avant d'importer cv2 :
* sudo apt-get update
* sudo apt-get install ffmpeg libsm6 libxext6 -y
* sudo apt-get install libgl1

In [3]:
import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image as im

In [11]:
from datetime import date

In [4]:
update_storage_access()

In [5]:
environment = get_environment()

root_path = get_root_path()
bucket = environment["bucket"]
path_s3_cayenne_data = environment["sources"]["PLEIADES"][2022]["guyane"]
path_local_cayenne_data = os.path.join(root_path, environment["local-path"]["PLEIADES"][2022]["guyane"])

bucket = environment["bucket"]

path_s3_pleiades_data_2022_guyane = environment["sources"]["PLEIADES"][2022]["guyane"]
path_s3_bdtopo_data_2022_guyane = environment["sources"]["BDTOPO"][2022]["guyane"]
path_local_pleiades_data_2022_guyane = environment["local-path"]["PLEIADES"][2022]["guyane"]
path_local_bdtopo_data_2022_guyane = environment["local-path"]["BDTOPO"][2022]["guyane"]

path_s3_pleiades_data_2022_martinique = environment["sources"]["PLEIADES"][2022]["martinique"]
path_local_pleiades_data_2022_martinique = environment["local-path"]["PLEIADES"][2022]["martinique"]

path_s3_pleiades_data_2017_martinique = environment["sources"]["PLEIADES"][2017]["martinique"]
path_local_pleiades_data_2017_martinique = environment["local-path"]["PLEIADES"][2017]["martinique"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

### Chargement données.

In [8]:
# DL PLEIADE guyane 2022
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2022_guyane}",
        lpath=f"../{path_local_pleiades_data_2022_guyane}",
        recursive=True)

In [9]:
# DL BDTOPO guyane 2022
fs.download(
        rpath=f"{bucket}/{path_s3_bdtopo_data_2022_guyane}",
        lpath=f"../{path_local_bdtopo_data_2022_guyane}",
        recursive=True)

In [6]:
# DL PLEIADE martinique 2022
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2022_martinique}",
        lpath=f"../{path_local_pleiades_data_2022_martinique}",
        recursive=True)

In [7]:
# DL PLEIADE martinique 2017
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2017_martinique}",
        lpath=f"../{path_local_pleiades_data_2017_martinique}",
        recursive=True)

In [10]:
#image choisie
filename = '../data/PLEIADES/2022/GUYANE/ORT_2022072050325085_0353_0545_U22N_16Bits.jp2'
date = datetime.strptime(re.search(r'ORT_(\d{8})', filename).group(1), '%Y%m%d')
date

In [11]:
image = SatelliteImage.from_raster(
        filename,
        date = date, 
        n_bands = 4,
        dep = "973"
    )
image.normalize()

In [12]:
np.shape(image.array.transpose())

## Idée globale : si le pixel est rouge, le changer en noir, le reste en blanc.
Pour cela, il faut fixer des seuils : quel seuil sur l'infrarouge choisir, quels nuances de rouges colorier en noir ou en blanc ?
Ici les pixels seront en décimal.

## Méthode 1 : Seuillage sur l'infrarouge.

Voici l'image en couleurs puis avec l'infrarouge à la place du rouge.

In [13]:
image.plot([0,1,2])
image.plot([3,1,2])

In [14]:
#on extrait l'array de l'image pour avoir les valeurs des pixels
img = image.array

#multiplication par 255 et convertion en uint8 pour avoir le bon format
img = (img * 255).astype(np.uint8)

img = img.transpose()

On choisit ce seuil en testant plusieurs quantiles et en regardant le meilleur résultat

In [15]:
np.quantile(img[:,:,3],0.5)

In [16]:
#on parcours tous les pixels et on les modifie en fonction du seuil
for row in range(img.shape[0]):
    for col in range(img.shape[1]):
        i = img[row,col,3]
        if i > 139: #médiane
            img[row, col] = np.array([0, 0, 0,0]) # blanc

        else : 
            img[row, col] = np.array([255,255,255,255]) # noir

In [17]:
img = img.transpose()

#On veut le bon format
img = (img/255).astype(np.float64)

image.array = img

In [18]:
image.plot([0,1,2])

## Méthode 2 : transformation de l'image infrarouge en image RGB en remplacant le rouge par l'infrarouge puis seuillage sur les 3 couleurs.

In [19]:
#on recharge l'image originale
image = SatelliteImage.from_raster(
        filename,
        date = date, 
        n_bands = 4,
        dep = "973"
    )
image.normalize()

On va refaire le même algorithme, sauf qu'on va changer les seuils sur les pixels RGB à la place de seulement infrarouge comme précedemment.

In [20]:
img = image.array

img = (img * 255).astype(np.uint8)


img = img.transpose()

* Etape 1 : Pour que le pixel soit une nuance de rouge, il faut d'abord vérifier si les valeurs verte et bleue sont proches.
* Etape 2 : On décide que les nuances de rouge très claires correspondent à des toits de bâtiments.
* Etape 3 : On fixe le seuil de la valeur rouge à 110 et la valeur de rouge avec une autre couleur doit être supérieure à 20 minimum.
* Etape 4 : Si le pixel ne vérifie aucune des étapes précédentes, alors il est colorié automatiquement en blanc.

Pour fixer ce seuil, des tests visuels ont été effectués et on a utilisé ce site https://htmlcolorcodes.com/fr/selecteur-de-couleur/ pour comprendre comment fonctionnent les nuances de rouge sur des pixels en décimal (et ce site pour comprendre les pixels en décimal en général http://www.proftnj.com/RGB3.htm).
L'algorithme qui suit met quelques secondes à tourner, il sera long s'il est appliqué sur l'ensemble des images de la base de données.
Pour mieux réussir à faire ce seuillage en étape 3, on devrait créer une fonction racine carré et prendre toutes les valeurs au dessus de la courbe.
Cette fonction est difficile à déterminer... Cependant, on peut la faire uniquement avec deux paramètres : la valeur du rouge et la valeur minimale entre le bleu et le vert. 

In [21]:
# On va parcourir tous les pixels de l'image
for row in range(img.shape[0]):
    for col in range(img.shape[1]):
        b = img[row,col, 1]
        g = img[row,col, 2]
        r = img[row,col, 3]
        mini = min(b,g)
        maxi = max(b,g)
        
        if maxi-mini <= 20 : #étape 1
            
            if r > 200 and mini >= 110 and r>= (20+mini): #étape 2
                img[row, col] = [255,255,255,255] # blanc
            elif  r>= (20+mini) and r >= 110: #étape 3
                img[row, col] = [0,0,0,0] #noir
            else : #étape 4
                img[row, col] = [255,255,255,255] # blanc

        else : #étape 4
            img[row, col] = [255,255,255,255] # blanc

In [22]:
img = img.transpose()

#On veut le bon format
img = (img/255).astype(np.float64)

image.array = img

In [23]:
image.plot([0,1,2])

Ici, la 2ème méthode est plus effective. En effet, on prend en compte que certains toits vont refleter les ondes infrarouges et donc sortir comme rouge clair une fois l'image convertie en RGB. Cependant, la méthode du seuillage sur l'infrarouge reste la plus simple : méthode concise et temps de calcul plus rapide. Les résultats semblent globalement bon dans les deux cas. A voir quelle méthode se généralise le mieux...

## Essai au cas où : Détection de contours

In [25]:
#On récupère l'image au format RGB puis on la transforme en png pour pouvoir l'utiliser avec OpenCV
img = image.array

img = img[[0,1,2],:, :]

img = (img*255).astype(np.uint8)

#convertir l'array en image PIL pour ensuite l'avoir en png
img = im.fromarray(img.transpose(1, 2, 0).astype('uint8'), 'RGB')

#enregistrer l'image en tant que fichier PNG pour pouvoir utiliser OpenCV
img.save('masque.png')

Afin de ne pas faire crasher le kernel, nous allons préalablement créer des fonctions d'affichage des images avec OpenCV (la fonction de base cv2.imshow() fait crasher le kernel)

In [26]:
#images en nuances de gris
def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_RGBA2BGRA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_RGB2BGR)

    return plt.imshow(a, **kwargs, cmap= "gray")

#images en couleurs
def cv2_imshow2(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_RGBA2BGRA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_RGB2BGR)

    return plt.imshow(a, **kwargs, cmap= "brg")

In [30]:
#on lit l'image avec OpenCV
img = cv2.imread('masque.png')

#avoir le bon format : ici on veut RGB (la transposée nous a fait passer en BGR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
# Find Canny edges
edged = cv2.Canny(gray, 100, 200)
  
# Finding Contours
# Use a copy of the image e.g. edged.copy()
# since findContours alters the image
contours, hierarchy = cv2.findContours(edged, 
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
img_contours = cv2.drawContours(img, contours, -1, (0, 255, 0), 5)
cv2_imshow2(img_contours)

Ce qui serait intéressant de faire, c'est de prendre en entrée des images où on sait qu'il y a des bidonvilles, on enlève la végétation avec la méthode 2 mais on garde les couleurs de base sur les pixels sans végétation et ensuite on classifie ces pixels avec un algo des k-means. Comme ça, on peut faire plusieurs classes d'objets : routes, gros bâtiments, maisons, bidonvilles. On determine à la main quelle classe correspond à quel type d'objets. Puis on fait un masque binaire : en blanc les pixels dans la classe bidonville, en noir le reste. Cela pourrait nous aider à constituer des données d'apprentissage.

## Maintenant, il serait intéressant de comparer deux images d'un même lieu sur deux périodes données où on sait qu'il y a eu une création/destruction de bâtiment entre temps.

## Idée globale : 
Trouver un lieu où il y a eu une construction/destruction de bâtiments entre deux millésimes et appliquer la méthode pour obtenir un masque sur les deux images infrarouge du même lieu à deux période données. Ensuite superposer ces deux masques et faire des différences de pixels : les différences sortiront en blanc et les similarités en noir. Pour finir, on veut lisser l'image, c'est à dire trouver un seuil à partir duquel on considère qu l'amas de pixels représente un changement significatif. 

La première difficulté ici est de trouver deux images différentes d'un même lieu. On va parcourir les données pour savoir si c'est possible ou si c'est irréalisable sans faire de transformation.

In [34]:
#image 1
filename_1 = '../data/PLEIADES/2022/MARTINIQUE/ORT_2022_0713_1607_U20N_8Bits.jp2'
date_1 = date.fromisoformat('2022-01-01')

In [35]:
image_1 = SatelliteImage.from_raster(
        filename_1,
        date = date_1, 
        n_bands = 3,
        dep = "972"
    )
image_1.normalize()

In [23]:
image_1.plot([0,1,2])

On créé des fonctions crs_to_gps() et gps_to_crs() pour retrouver l'image sur google maps grâce à ses coordonnées crs transformées en gps et inversement, pour retrouver un point gps repéré sur Google Maps et le situer sur les images de la base de données grâce à ses données crs.

In [57]:
import re
import pyproj

def crs_to_gps(satellite_image) :
    
    #avoir les coordonnées du boundingbox
    boundingbox = list(satellite_image.bounds)
    
    #avoir le crs
    str_crs = str(satellite_image.crs)

    # Rechercher la séquence d'entiers dans la chaîne de caractères
    crs = re.findall(r'\d+', str_crs)
    crs = int(crs[0])
    
    #convertir en coordonnées gps
    src_proj = pyproj.Proj(crs)
    dest_proj = pyproj.Proj(proj='latlong')
    
    # Convertir les coordonnées dans le système de coordonnées de départ en coordonnées dans le système de coordonnées de destination (GPS)
    #on prend la coordonnées top-left arbitrairement
    x = boundingbox[0] #left
    y = boundingbox[3] #top
    lon, lat = pyproj.transform(src_proj, dest_proj, x, y, always_xy=True)
    
    # Retourner les coordonnées GPS (latitude, longitude)
    return lat, lon

In [58]:
crs_to_gps(image_1)

In [ ]:
def gps_to_crs(lat,lon,crs) :
    
    # Définir les systèmes de coordonnées de départ et d'arrivée
    src_proj = pyproj.Proj(proj='latlong')
    dest_proj = pyproj.Proj(crs)
    
    # Convertir les coordonnées GPS en coordonnées dans le système de coordonnées de destination (CRS)
    x, y = pyproj.transform(src_proj, dest_proj, lon, lat)
    
    # Retourner les coordonnées dans le CRS spécifié
    return x, y

#ca serait bien de faire une fonction qui cherche dans l'ensemble des données l'image qui contiendra cette coordonées.

(14.618229, -61.102878) correspond à des coordonnées gps d'un quartier avec des bâtiments à Schoelcher.
Essayons de retrouver l'image qui le contient.

Crééons une fonction qui réunie toutes les images de la base de données de la Martinique 2022, qui affiche l'ensemble de la Martinique et qui peut situer une coordonnées gps/crs dans l'ensemble et nous dire quelle image est concernée.